In [0]:
dbutils.fs.rm("dbfs:/FileStore/walmart/dataset/sample_dataset.dat")

Out[5]: True

# Prerequsite

## Load dataset into DataFrame

In [0]:
df = spark.read.csv(path="dbfs:/FileStore/walmart/dataset/sample_dataset.dat",
                    header=True,
                    inferSchema=True)
df.display()                    

ID,Name,Email,Phone,Date,State,City
1,John Doe,john.doe@example.com,(555) 456-7890,2025-01-01,CA,Los Angeles
2,Jane Smith,null,0987654321,2024-12-15,null,San Francisco
3,Sam Brown,sam.brown@,(555) 456-7890,null,NY,Albany
4,null,emma.wilson@example.com,(555) 456-7890,2025-01-10,TX,Dallas
5,Alex Johnson,alex.johnson@example.com,(555) 456-7890,2025-01-05,CA,San Diego
6,Michael Lee,michael.lee@example.com,null,2023-12-30,FL,Miami
7,Laura Kim,null,abcdefg123,2025-01-15,TX,Houston
8,Sophia Brown,sophia.brownexample.com,(555) 456-7890,2025-02-01,NY,New York
9,,mark.jones@example.com,(555) 456-7890,2023-11-20,null,Jacksonville
10,Chris Green,chris.green@example.com,(555) 456-7890,null,NY,null


# Data Quality

## Completeness

### Calcuting for single column

In [0]:
from pyspark.sql.functions import *

column_name = "Name"
non_null_count = df.filter(col(column_name).isNotNull()).count()
total_count = df.count()
completeness_score = ((non_null_count) / total_count) * 100
print(f"Completeness score for column:{column_name} is {completeness_score}")

Completeness score for column:Name is 80.0


### Calcuting for all columns

In [0]:
for column in df.columns:
    print(column)

ID
Name
Email
Phone
Date
State
City


In [0]:
completeness_score = {
    column:(df.filter(col(column).isNotNull())).count() / total_count for column in df.columns
}

for column, score in completeness_score.items():
    print(f"Completeness score for {column}: {score*100}%")

{'ID': 1.0, 'Name': 0.9, 'Email': 0.8, 'Phone': 0.9, 'Date': 0.8, 'State': 0.8, 'City': 0.9}
Completeness score for ID: 100.0%
Completeness score for Name: 90.0%
Completeness score for Email: 80.0%
Completeness score for Phone: 90.0%
Completeness score for Date: 80.0%
Completeness score for State: 80.0%
Completeness score for City: 90.0%


## Accuracy

In [0]:
df.select(
    (
        count(
            when(col("Phone").rlike(r"^\(\d{3}\) \d{3}-\d{4}$"),"True") / total_count
        )
        * 100
    ).alias("Phone")
).display()

Phone
0


In [0]:
from pyspark.sql.functions import col, count, when

df.select(
    (
        count(when(col("Phone").rlike(r"^\(\d{3}\) \d{3}-\d{4}$"), 1)) / total_count * 100
    ).alias("PhonePercentage")
).show()

+---------------+
|PhonePercentage|
+---------------+
|           70.0|
+---------------+



## Conformity

In [0]:
states = ["CA", "NY"]
df.select(
    ((count(when(col("State").isin(states), "State")) / total_count) * 100).alias(
        "State_Conformity"
    )
).display()

State_Conformity
50.0
